In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!git clone https://github.com/PotatoSpudowski/S.I.D.E.git
%cd S.I.D.E

Cloning into 'S.I.D.E'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 90 (delta 34), reused 76 (delta 20), pack-reused 0
Unpacking objects: 100% (90/90), done.
/content/S.I.D.E


In [0]:
!pip install efficientnet
!pip install annoy

     |████████████████████████████████| 645kB 2.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=297356 sha256=95b9ae102a984e51c8b9bbd5ae8eadb94aee4fba4e8bca9dee8368c4f37f6be6
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [5]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp '/content/drive/My Drive/GCC/Ignore_list.csv' 'Data/'
!cp '/content/drive/My Drive/GCC/val_data.csv' 'Data/'
!cp '/content/drive/My Drive/GCC/validation.zip' 'Data/'

In [7]:
!unzip 'Data/validation.zip'

Streaming output truncated to the last 5000 lines.
  inflating: validation/43FB519200000578-4856452-image-a-34_1504739283798.jpg  
  inflating: validation/relief-of-akhenaten-and-nefertiti-under-the-rays-of-the-sun-god-aten-bhffeb.jpg  
  inflating: validation/model-walks-the-runway-during-the-guo-pei-spring-summer-2017-show-as-picture-id632719860  
  inflating: validation/tumblr_nmpm2vr94s1sr18e6o1_1280.jpg  
  inflating: validation/jamie-oneal-during-the-36th-annual-academy-of-country-music-awards-picture-id104723023  
  inflating: validation/sandy-beach-at-the-spanish-seaside-resort-at-la-cala-de-mijas-ebfwrt.jpg  
  inflating: validation/dsc_0089.jpg  
  inflating: validation/stock-photo-grey-background-with-the-silhouette-of-cross-169211798.jpg  
  inflating: validation/period-house-and-trees-covered-in-snow-a-winter-scene-bm1jrn.jpg  
  inflating: validation/woman-rides-on-the-car-at-high-speed-f1mhtg.jpg  
  inflating: validation/beautiful-female-friends-applying-makeup-in-a-bea

In [0]:
import os
import sys

sys.path.append('..')

import tensorflow as tf
from model.model import *
from index_search.index_search import *
from utils.utils import *

In [0]:
df_path = "Data/val_data.csv"
ignore_df_path = "Data/Ignore_list.csv"

In [0]:
df = clean_data(df_path, ignore_df_path)

In [0]:
model = get_sentence_encoder(4)

In [0]:
def generate_image_embedding_pair(df, model):
    images, captions = df["images"], df["captions"]
    embeddings = generate_sentence_embeddings(captions, model)
    embeddings2 = []
    for embedding in tqdm(embeddings):
        embeddings2.append(tf.convert_to_tensor(np.reshape(embedding, (1, 512)), np.float32))

    return list(zip(images, embeddings2))

In [16]:
mapping = generate_image_embedding_pair(df, model)

100%|██████████| 13986/13986 [00:01<00:00, 9335.89it/s]


In [17]:
print("Image: ", mapping[0][0])
print("Embedding: ", mapping[0][1])

Image:  validation/Clearwater-Crossing-Shopping-Center-sign-Indianapolis-Indiana.jpg
Embedding:  tf.Tensor(
[[ 2.53905877e-02  3.21025848e-02 -5.45074753e-02  4.68080444e-03
  -8.16693529e-03  1.54621778e-02 -4.27414738e-02  9.98058766e-02
   2.48788446e-02 -4.22965437e-02  2.98991930e-02 -7.96434283e-02
   3.78425717e-02 -3.01000737e-02  5.51758856e-02 -6.84034154e-02
   8.25428125e-03 -5.08229695e-02  8.51492509e-02  2.96024010e-02
  -8.04320071e-03  5.91733530e-02  7.06206560e-02 -8.26347843e-02
   7.57722706e-02  5.09514986e-03  4.22729030e-02  2.40280014e-02
   2.55803317e-02 -7.24745542e-02 -2.66882237e-02  6.59405962e-02
  -1.17885033e-02 -6.86409026e-02 -9.40490440e-02 -5.67553714e-02
   6.90633804e-02 -2.13237777e-02  6.21191971e-02 -5.10851368e-02
  -2.74347309e-02 -5.81094772e-02  2.34503485e-02 -6.52914271e-02
  -6.72279224e-02 -4.97982204e-02  7.78026413e-03 -5.22024045e-03
   9.54101458e-02 -2.88248323e-02 -5.51740862e-02 -3.31830010e-02
   5.78102358e-02 -2.97407787e-02 

In [0]:
import pickle

In [0]:
with open("val_data_mapping.pkl", "wb") as fp:
    pickle.dump(mapping, fp)

In [0]:
!cp 'val_data_mapping.pkl' '/content/drive/My Drive/GCC/' 